# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:

upgraded on 2024-07-21
```Python
!pip install crewai==0.41.1 crewai_tools==0.4.26 langchain_community==0.2.9  
```

In [1]:
!pip show crewai

Name: crewai
Version: 0.41.1
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /home/gongai/anaconda3/envs/crewai/lib/python3.11/site-packages
Requires: appdirs, click, embedchain, instructor, json-repair, jsonref, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: arxiv, newsletter_gen, resumer


In [2]:
!pwd

/home/gongai/projects/wgong/py4kids/lesson-18-ai/crewai/gongai


In [3]:
from IPython.display import Markdown, display
import os
from time import time
import json
from pprint import pprint

# import crewai   # Version: 0.30.11
from crewai import Agent, Task, Crew, Process

from crewai_tools import (
    SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, MDXSearchTool,
    DirectoryReadTool, FileReadTool, BaseTool
)

from langchain_openai import ChatOpenAI

from pydantic import BaseModel

from api_key_store import ApiKeyStore


# Warning control
import warnings
warnings.filterwarnings('ignore')

/home/gongai/anaconda3/envs/crewai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from helper import *

## Setup

### Inputs

In [5]:
file_suffix = "t6"  
model_name = "gpt-4o-mini" # "gpt-3.5-turbo"
# see list of OpenAI models

# inputs json doc
# 
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/crewAIInc/crewAI',
    'personal_writeup': """
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data science. Noah has successfully led
        major tech initiatives and startups, proving his ability to drive innovation and growth in the tech industry. 
        Ideal for leadership roles that require a strategic and innovative approach.
    """
}


### Outputs

In [6]:
folder_name = "job-app"
create_folder(folder_name)

In [7]:
filein_resume = Path(folder_name) / f'orig_resume-t2.md'
fileout_research = Path(folder_name) / f"task_research-{file_suffix}.md"
fileout_profile = Path(folder_name) / f"task_profile-{file_suffix}.md"
fileout_resume = Path(folder_name) / f"task_resume-{file_suffix}.md"
fileout_interview = Path(folder_name) / f"task_interview-{file_suffix}.md"

In [8]:
str(filein_resume), str(fileout_research), str(fileout_profile), str(fileout_resume), str(fileout_interview), 

('job-app/orig_resume-t2.md',
 'job-app/task_research-t6.md',
 'job-app/task_profile-t6.md',
 'job-app/task_resume-t6.md',
 'job-app/task_interview-t6.md')

### LLM and keys

In [9]:
s = ApiKeyStore()

os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI/Yiwen")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

os.environ["OPENAI_MODEL_NAME"] = model_name   # gpt-4 was default, throws rate-limit error 

## Define Tools

In [10]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path=filein_resume)
semantic_search_resume = MDXSearchTool(mdx=str(filein_resume))

## Create Agents

In [11]:
# Agent 1: Researcher
researcher = Agent(
    role="Job Researcher",
    goal="Make sure to do amazing analysis on  job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory="""
        "As a Job Researcher, your prowess in  navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary  qualifications and skills sought "
        "by employers, forming the foundation for  effective application tailoring."
    """.replace('"',''),
)

In [12]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Job applications",
    goal="Do increditble research on job applicants  to help them stand out in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Equipped with analytical prowess, you dissect  and synthesize information "
        "from diverse sources to craft comprehensive  personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    """.replace('"',''),
)

In [13]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist",
    goal="Find all the best ways to make a  resume stand out in the job market.",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "With a strategic mind and an eye for detail, you  excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they  resonate perfectly with the job's requirements."
    """.replace('"',''),
)

In [14]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Job Interview Preparer",
    goal="Create interview questions and talking points  based on the resume and job requirements",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose=True,
    backstory="""
        "Your role is crucial in anticipating the dynamics of  interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success,  ensuring they can confidently address all aspects of the "
        "job they are applying for."
    """.replace('"',''),
)

## Create Tasks

In [15]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description="""
        "Analyze the job posting URL provided ({job_posting_url})  to extract required key skills, experiences, and qualifications. "
        "Use the tools to gather content and identify  and categorize the requirements."
    """.replace('"',''),
    expected_output="A structured list of job requirements, including necessary skills, qualifications, and experiences.",
    output_file=str(fileout_research),
    agent=researcher,
    async_execution=True
)

In [16]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description="""
        "Compile a detailed personal and professional profile  using the GitHub ({github_url}) URLs, "
        "and personal write-up ({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    """.replace('"',''),
    expected_output="""
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and communication style."
    """.replace('"',''),
    output_file=str(fileout_profile),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [17]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
# depends on research_task, profile_task
resume_strategy_task = Task(
    description="""
        "Using the profile and job requirements obtained from previous tasks,"
        " tailor the resume to highlight the most relevant areas. "
        "Employ tools to adjust and enhance the resume content. "
        "Make sure this is the best resume even but don't make up any information. "
        "Update every section,  inlcuding the initial summary, work experience, skills, and education. "
        "All to better reflrect the candidates  abilities and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    """.replace('"',''),
    output_file=str(fileout_resume),
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [18]:
# Task for Interview Preparer Agent: Develop Interview Materials
# depends on research_task, profile_task, resume_strategy_task
interview_preparation_task = Task(
    description="""
        "Create a set of potential interview questions and talking  points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion points. "
        "Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume  and how it matches the job posting."
    """.replace('"',''),
    expected_output="""
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    """.replace('"',''),
    output_file=str(fileout_interview),
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Create the Crew

In [19]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Run the Crew Workflow

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
### this execution will take a few minutes to run
ts_start = time()
result = job_application_crew.kickoff(inputs=job_application_inputs)
ts_stop = time()

 [2024-07-21 13:04:27][DEBUG]: == Working Agent: Job Researcher
 [2024-07-21 13:04:27][INFO]: == Starting Task: 
        Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund)  to extract required key skills, experiences, and qualifications. 
        Use the tools to gather content and identify  and categorize the requirements.
    
 [2024-07-21 13:04:27][DEBUG]: == Working Agent: Personal Profiler for Job applications
 [2024-07-21 13:04:27][INFO]: == Starting Task: 
        Compile a detailed personal and professional profile  using the GitHub (https://github.com/crewAIInc/crewAI) URLs, 
        and personal write-up (
        Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in
        managing remote and in-office teams, and expert in multiple programming languages and frameworks. 
        He holds an MBA and a strong background in AI and data

In [21]:
print(f"\n\nElapsed time: {(ts_stop-ts_start):.2f} sec")   # Elapsed time: 53.83 sec

# 26.14 sec on 2024-06-29

# 21.30 sec on 2024-07-21

# 29.99 sec on 2024-07-21 (after upgrading crewai==0.41.1

# 354.39 sec on 2024-07-21 (using GPT-4: search is more extensive, taking much longer)

# 66.60 sec on 2024-07-21 (using gpt-4o-mini)

# 78.26 sec on 2024-07-21 (using gpt-4o-mini)



Elapsed time: 78.26 sec


## Display original resume

In [22]:
display(Markdown(filein_resume))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Display revised resume

In [23]:
display(Markdown(fileout_resume))

# Noah Williams
- **Email**: noah.williams@example.dev
- **Phone**: +44 11 111 11111
- **LinkedIn**: linkedin.com/in/noah-williams

## Profile
Results-driven Full Stack Engineer with 18 years of experience in software development, specializing in both front-end and back-end technologies. Proven ability to design and implement scalable applications and APIs using languages such as Ruby, Python, JavaScript, and TypeScript. Extensive experience in leading cross-functional teams and managing complex projects from ideation to deployment in both remote and in-office settings. Strong expertise in AI technologies, data protection, and user-centered design, with a passion for enhancing user experiences and driving innovation in tech environments.

## Skills
- **Languages**: JavaScript, Python, Ruby, TypeScript, Elixir
- **Frameworks**: React, Angular, Node.js, Ruby on Rails
- **Databases**: MySQL, MongoDB
- **Web Development**: HTML, CSS, RESTful APIs, UI/UX design
- **DevOps**: CI/CD, Git, cloud services (AWS, Azure)
- **Soft Skills**: Excellent communication, project management, problem-solving, leadership

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a key revenue pillar, leading to a substantial increase in customer acquisition and retention.
- Integrated cutting-edge AI technologies and scalable databases to enhance product capabilities.
- Spearheaded the development of multiple web applications, ensuring high responsiveness and efficiency through rigorous testing and optimization.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy and operations, collaborating with C-level executives to shape the technological vision.
- Managed diverse teams across North America and Europe, fostering a culture of innovation and respect.
- Enhanced team performance through mentoring, resulting in improved project delivery timelines.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded a startup focused on IoT solutions, overseeing product development for a GPS tracking collar that improved pet safety.
- Established production facilities and navigated the company through initial funding, demonstrating strong leadership and entrepreneurial skills.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Led strategic initiatives to enhance collaboration and productivity within engineering teams.
- Developed and launched new products using Ruby on Rails and React, significantly improving user engagement and satisfaction.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Built a high-performance engineering team that achieved a 500% increase in revenue.
- Pioneered data analytics integration into business strategies, enhancing decision-making processes.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized core APIs used by thousands of users, significantly enhancing system functionality and performance.
- Implemented caching strategies that improved application responsiveness.

### BetCraft: CTO — 2013 - 2015
- Led technology strategy post-Series A funding, driving significant advancements in platform capabilities.
- Collaborated with cross-functional teams to develop scalable applications using React and Angular.

## Education

### MBA in Information Technology
London Business School

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh

## Projects & Contributions
- **CrewAI**: Contributed to the development of a framework for orchestrating role-playing autonomous AI agents, enhancing collaborative intelligence.
- **InnovPet GPS Tracking Collar**: Developed a groundbreaking product that increased safety for pets and peace of mind for owners, incorporating advanced data management techniques.

## Interests
- Passionate about AI advancements and their applications in technology.
- Dedicated to mentoring upcoming engineers and fostering a continuous learning environment.

## Communication Style
Noah is known for his articulate communication style, adept at translating complex technical concepts into accessible language. His leadership emphasizes transparency and collaboration, creating an inclusive environment conducive to innovation.

This resume strategically highlights Noah's qualifications and experiences relevant to the Full Stack Engineer role, showcasing his extensive technical expertise and leadership capabilities.

In [24]:
display(Markdown(fileout_interview))

### Interview Preparation Document for Noah Williams - Full Stack Engineer Position at AI Fund

#### Key Interview Questions

1. **Experience with Full Stack Development**
   - Can you describe your experience as a Full Stack Developer and the types of applications you have built?
   - How do you ensure both the front-end and back-end of your applications are well-integrated?

2. **Working with Development Teams**
   - How have you collaborated with product managers and development teams in previous roles?
   - Can you provide an example of a successful project where you played a key role in ideating software solutions?

3. **Front-End Development**
   - What front-end technologies are you most proficient in, and how have you utilized them in your projects?
   - How do you approach visual design to create appealing user interfaces?

4. **Database Management**
   - Can you discuss your experience managing databases? Which databases are you most comfortable with, and why?
   - How have you implemented data protection settings in your applications?

5. **API Development**
   - Describe your experience in writing and managing APIs. What best practices do you follow to ensure they are effective and efficient?
   - Can you share a specific example where your API significantly improved application functionality?

6. **Software Testing and Debugging**
   - What strategies do you employ to test software for responsiveness and efficiency?
   - How do you handle troubleshooting and debugging in your development process?

7. **AI and Data Technologies**
   - Can you elaborate on your experience with AI technologies and how you've integrated them into your projects?
   - How do you leverage data analytics in your decision-making processes?

8. **Leadership and Communication Skills**
   - How do you foster collaboration and respect within your engineering teams?
   - Can you share an experience where your communication skills helped resolve a technical challenge?

9. **Project Management**
   - Describe how you manage complex projects and ensure timely delivery.
   - What project management tools or methodologies have you found most effective in your past roles?

10. **Adaptability in Early-stage Environments**
    - How do you adapt your working style to fit an early-stage environment, especially when resources are limited?
    - Can you provide an example of a time when you had to take initiative to get a project done?

#### Talking Points

- **Proven Leadership in Software Engineering**: Highlight your role as Director of Software Engineering at DataKernel, where you transformed the engineering division into a key revenue pillar while integrating cutting-edge AI technologies.

- **Extensive Technical Expertise**: Emphasize your proficiency in multiple programming languages and frameworks (Ruby, Python, JavaScript, React, Angular, Node.js) and how this aligns with the requirements for the Full Stack Engineer position.

- **Successful Project Management**: Discuss your experience managing cross-functional teams and your ability to deliver results aligned with strategic goals, as evidenced by your work at InnovPet and PrintPack.

- **Innovative Solutions**: Mention your entrepreneurial background as the founder of InnovPet, emphasizing your ability to develop and launch innovative IoT solutions, such as the GPS tracking collar for pets.

- **Strong Communication Skills**: Stress your ability to translate complex technical concepts into accessible language, fostering a collaborative environment in diverse teams.

- **Passion for AI Technologies**: Share your enthusiasm for emerging AI technologies and their applications, particularly how you integrated them into your engineering practices.

- **Ownership and Accountability**: Illustrate your characteristic ownership orientation by providing examples of how you have taken responsibility for project outcomes and driven significant results in your teams.

#### Conclusion
Preparing for these questions and talking points will help Noah Williams effectively convey his qualifications and how they align with the Full Stack Engineer role at AI Fund. Emphasizing his extensive experience, technical skills, and leadership capabilities will position him as a strong candidate for the position.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)